# Tutorial with UDF -  Michelle Pichardo  

This tutorial shows the basic steps of using SEP to detect objects in an image and preform some basic photometry. 
* Photometry: 
> a branch of science that deals with measurement of the intensity of light. 

# Imort Packages 
* Numpy:(Package) element-by-element operations- used for speed
* SEP: Python lib for Source Extraction and Photometry 
> * Command-line program for segmentation and analysis of astronomical images. Reads FITS files, preofroms several tasks. 
> https://sep.readthedocs.io/en/v1.0.x/index.html
* astropy.io:(Package) provides access to FITS files 
> * Flexible Image Transport System
> > A portable file standard used in astronomy community to store images and tables
* Matplotlib: Lib for creating visualizations with python
> * rcParams: Matplotlib defines rc(runtime configuration)containing default styles for every plot element created. The configurations can be modified with the rc parameters.
> https://www.data-blogger.com/2017/11/15/python-matplotlib-pyplot-a-perfect-combination/
* %matplotlib inline: reders the figure in a notebook instead of displaying a figure. 


In [ ]:
#packages are used to read the test image
#and display plots 

import numpy as np 
import sep 
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams 
%matplotlib inline


#from astropy.io.fits import getdata (not used)


#Setting the parameters for all future figures: 
#selecting figure function and figsize argument 
#figsize takes(float,float)=(width,height) in inches 
#w= 10.in h=8.in 
#default was: 6.4,4.8 


rcParams['figure.figsize'] = [10.,8.] 

In [ ]:
check_info_to_convertAB = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
hdu_list = fits.open(check_info_to_convertAB)
hdu_list.info()
image_data = hdu_list[0].data

In [ ]:
image_header = hdu_list[0].header
print(image_header)

# Read an example image from a FITS file and display it

Information on the following blocks: 
* Verify the image is 256 x 256 pixels 
* More info: https://docs.astropy.org/en/stable/generated/examples/io/plot_fits-image.html#sphx-glr-generated-examples-io-plot-fits-image-py
* ext = 0 
> * This argument calls the header
> * Without other argumentst it would also only call the header 
* fits.getdata: 
> * returns: array, record array or groups of data object
> https://docs.astropy.org/en/stable/io/fits/api/files.html#astropy.io.fits.getdata

In [ ]:
#call image
data = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz .fits', ext =0)

#verify the shape: 
print("Our array is a 2-D array with the following elements: ")
print(data.shape)

#checking type: 
#print(type(data))

# Show the image 

Information on the following blocks: 
* np.mean and np.std: 
> Calls the mean value and standard div from a set of values 
* plt.imshow: 
> Functions with certain parameters,(X, interpolatoin, cmap, vmin, vmax, origin)
> * https://www.geeksforgeeks.org/matplotlib-pyplot-imshow-in-python/
> * The input may either be actual RGB(A) data, or 2D scalar data
* X: is the data of the image 


* cmap: is a color map instance (selection of colors) 
> https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
* vmin,vmax: the color bar range 
* interpolation: interpolation used to display an image 
* origin: place the [0,0] index in a corner of axes 

In [ ]:
#check the mean and std values 

print('Mean values from data: ')
print(np.mean(data))
print('\nStandard Deviation values from data: ')
print(np.std(data))

In [ ]:
#assign the mean and standard deviation to variables 

m, s = np.mean(data), np.std(data)

plt.imshow(data, interpolation='nearest', cmap='gray', 
           vmin=m-s, vmax=m+s, origin='lower')

plt.colorbar()


# Background subtraction 
* This step is needed before sources can be detected 
* In SEP, background estimation and source detection are two seperate steps 
* sep.background: 
> Representation of spatially variable image background and noise 
> * arguments(data)- 'not the same as our data' is a 2-d array
> * methods: background, background rms, subfrom 
> * https://sep.readthedocs.io/en/v1.0.x/api/sep.Background.html
> * returns: rms, an array with same dimensions as original 

In [ ]:
#measure a spatially varying background on the image 

#assign the background data to a variable 
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

### sep.Background(): 
* returns an Background object that holds information on the spatially varying background and spatially varying background noise level. 

### <span style = 'color:red'> The Values do not match the Tutorial </span>
* Confirmed by prof, this is ok 

In [ ]:
#get a 'global' mean and noise of the image backgroung: 

# print the global background level 
print(bkg.globalback) 

# print the global background Root Mean Square 
print(bkg.globalrms)


In [ ]:
# evaluate background as 2-d array, same size as original image

#set background data to a 2-d array 
bkg_image = np.array(bkg)

In [ ]:
# show the background

plt.imshow(bkg_image, interpolation='nearest', 
           cmap='gray', origin='lower')
plt.colorbar()


In [ ]:
# evaluate the background noise as 2-d array, same size as original image 

#Modify the background data to be the rms of the background (noise) 
bkg_rms = bkg.rms()

In [ ]:
#show the noise (bkg_rms)

plt.imshow(bkg_rms, interpolation='nearest', 
           cmap='gray', origin='lower')
plt.colorbar()


In [ ]:
# subtract the background 
#data(total info) - bkg(noise)

data_sub = data - bkg

# Object detection 
Now that we've subtracted the background, we can run object detection on the background subtracted data. You can see the background noise is pretty flat. So we're setting the detection threshold to be a constant value of 1.5σ  where σ is the global background RMS. 

* set.extract: 
> Extracts sources from an image 
> arguments: (data,thresh, err) 
> https://sep.readthedocs.io/en/v1.0.x/api/sep.extract.html
* Data: 2-d array 
* Thresh: float, is the threshold value for detection. 
* err: error or variance, this can be used to specify a pixel by pixel detection threshold 
* returns: Extracted object parameters
> Threshold at object location, err second moment errors 

In [ ]:
# extract data, set threshold ot 1.5 

objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

#view an entry and it's x,y  positions 
#print(objects[0])
#print(objects['x'])
#print(objects['y'])

# Length of objects found 


In [ ]:
# how many objects were detected 
print("The amount of sources found: "+ str(len(objects)) )


### objects['x'] and objects['y'] will give the centroid coordinates of the objects. Just to check where the detected objects are, we'll over-plot the object coordinates with some basic shape parameters on the image: 

# About the following block: 
* matplotlib.patches: 
> patches contain classes to pull from
> https://matplotlib.org/api/patches_api.html
> Ellipse: is a class in patches, a scale free ellipse 
* Ellipse [xy, width, hight[,angle])
> https://matplotlib.org/api/_as_gen/matplotlib.patches.Ellipse.html#matplotlib.patches.Ellipse

* plt.subplots(): 
> Creates a figure and a set of subplots 
> * Returns: Fig: figure and ax: axes.Axes object or array of Axes objects 
> https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html
* Key to showing image is Axes(ax) 
> * as.imshow(): 
>> * Display data as an image, on a 2-D raster(bitmap image- ind. pixels as a square) 
>> The input is a 2D scalar daata, which will be rendered as a pseudocolor image. The number of pixels used to render an image is set by the axes size and the dpi(dots per inch) of the figure 
>> * vmin and vmax are related to the cmap (colormap) where m:mean and s:standard div

In [ ]:
# import package for displaying ellipses 
from matplotlib.patches import Ellipse

# plot background-subtracted image
# define the figure and axes 
fig, ax  = plt.subplots()

# define the mean and standard div values 
#from the new background without noise
m, s = np.mean(data_sub), np.std(data_sub)


# Display the Axes defined by our our array data_sub
#define the color scale by using the mean and standard div 
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')


# plot an ellipse for each object
for i in range(len(objects)):             #loop through [0,69-1] total of 69
    
    #define the ellipse 
    #xy = (x,y) takes the first elements and assigns the coordinate
    #width= total length of the horizontal axis 
    #height = total length of the vertical
    #angle = rotation anticolockwise
    #facecolor= no fill
    #edge = parimeter 
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('yellow')
    ax.add_artist(e)



In [ ]:
# available fields 
#these are other data types within objects that we can use 
objects.dtype.names

# Aperture Photometry 
Finally, we'll preform simple circular aperture photometry with a 3 pixel radius at the locations of the objects: 
* measurement of brightness in the aperture 
* We kept the parameters from the tutorial

# Important notes 

See image below: The imformation extracted is from the FITS Header
Steps taken: 

![Image of FITSHeader information](FITSHeader.jpg)

1. We originally went to the trouble of intalling some packages in this website
> * https://www.stsci.edu/hst/instrumentation/acs/data-analysis/zeropoints
2. We were under the impression it was necessary to do this and gathered information on the functions 
> * https://acstools.readthedocs.io/en/latest/acszpt.html
3. We noticed our Detector and filter are not used in these newer packages 
> * there are explicit messages noting only specific filter and intruments would work. 
4. We then found a website specific to the IR F105W 

![Image of F105AB info: ABmag= 26.2687](F105WABmag.jpg)

> * this gave specifc values for ABmag
> * https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration#section-cc19dbfc-8f60-4870-8765-43810de39924

5. We needed to account for the correction from .2" to infinity according to STscI 
> * Another issue arose, the website did not have a value for F105W only F105LB. We could not find any other figures with this data. We aproximated the error to .877 
> * https://www.stsci.edu/hst/instrumentation/acs/data-analysis/aperture-corrections
6. Since we were unable to use the function specific to STScI we opted to take the values given and form the equation provided. 

![Image of STScI code](STScIcode.jpg)

7. We noticed issues in the log10 function and ran a for loop to adjust any negative values. I'm not too confortable with flux but I assumed the negative was relative. 


In [ ]:
# Compute initial flux 

#flux, fluxerr, and flag are all 1d arrays with one entry per object
#this computes the sum of pixels withing the given radius 
#our error or variance is Bkg (the noise) rms


flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#Check to see all objects have thier flux values: 

#for i in range(len(objects)):
    #print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))
    
    
#Only print 10, as per the tutorial 
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# Convert flux to AB magnitude 
* From objects = sept.extract 
> * flux: sum of member pixels in unconvoled data (not connected) 

In [ ]:
#correct the Flux array values 

correction_inf = 0.877
flux_inf = flux/correction_inf

#convert instrumental fluxes to physical fluxes and magnitudes 
# using a for loop to account for negative values 

for i in range(len(flux_inf)): 
    if flux_inf[i]> 0: 
        n = -2.5 * np.log10(flux_inf[i]) + 26.2687
        flux_inf[i] = n 
        #print(n)
    elif flux_inf[i]<0: 
        n = -2.5 * np.log10(-flux_inf[i]) + 26.2687
        flux_inf[i] = n 
        #print(n)



In [ ]:
#Check to see all objects have thier flux values: 
#for i in range(len(objects)):
    #print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))
    
    
#Only print 10, as per the tutorial 
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux_inf[i], fluxerr[i]))

# Differences in graphs 
* we noticed differneces in our histograms 
* Depening on the method chosen the histograms will differ, the method used for this one is described above. 

In [ ]:
#Create the histogram
histogram = plt.hist(flux_inf, bins='auto')